<p><img alt="Colaboratory logo" height="45px" src="/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

#<h1>Import packages</h1>

In [ ]:
import easydict
import os
import numpy as np
import math
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision

import pandas as pd
from tqdm import tqdm 
import time

from random import randint
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation
from IPython.display import HTML
from matplotlib import rc
%matplotlib inline

import copy as cp
from IPython.display import clear_output

 
use_cuda = True
if torch.cuda.is_available():
  device = torch.device("cuda")


np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%.3g" % x))

cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

<p><img alt="Colaboratory logo" height="45px" src="/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

#<h1>Control Data form</h1>

In [ ]:
TerrainData = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Quadruped/01.Motion_data_flat.txt')
# TerrainData = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Quadruped/01.Motion_data_step.txt')
TerrainData

,PathNum,NodeNum,baseX,baseY,baseZ,baseR,baseP,baseYaw,base_dX,base_dY,base_dZ,base_dR,base_dP,base_dYaw,P_LFx,P_LFy,P_LFz,P_RFx,P_RFy,P_RFz,P_LHx,P_LHy,P_LHz,P_RHx,P_RHy,P_RHz,F_LFx,F_LFy,F_LFz,F_RFx,F_RFy,F_RFz,F_LHx,F_LHy,F_LHz,F_RHx,F_RHy,F_RHz,LF,RF,LH,RH
0,0,1,0.001688,-0.001070,0.123544,-0.003039,0.000163,0.002243,0.041294,-0.024394,0.003102,-0.022179,-0.001846,0.007847,0.087072,0.054000,3.751100e-39,0.087072,-0.054000,0.000000,-0.087072,0.054000,0.000047,-0.087072,-0.054000,3.159170e-40,2.581980,-2.76937,8.42542,0.783676,-0.92922,13.5573,2.622500,-1.83203,21.3714,2.212310,-3.35421,7.5617,0,0,0,0
1,0,2,0.005097,-0.003231,0.373127,-0.009178,0.000494,0.006774,0.124717,-0.073675,0.009370,-0.066987,-0.005575,0.023701,0.262976,0.163091,1.275130e-39,0.262976,-0.163091,0.000000,-0.262976,0.163091,0.000142,-0.262976,-0.163091,1.073910e-40,7.798090,-8.36404,25.44650,2.366860,-2.80643,40.9458,7.920480,-5.53310,64.5460,6.681620,-10.13040,22.8379,0,0,0,0
2,0,3,0.006932,-0.004783,0.502204,-0.013175,0.000654,0.009759,0.182454,-0.107737,0.011312,-0.050916,-0.011678,-0.031935,0.353566,0.219273,0.000000e+00,0.353566,-0.219273,0.000000,-0.353566,0.219273,0.000145,-0.353566,-0.219273,0.000000e+00,8.366870,-11.71000,34.86590,2.944270,-3.63719,48.8534,11.992100,-8.74563,85.3883,7.137020,-13.38440,30.6945,1,1,1,1
3,0,4,0.007096,-0.005688,0.503320,-0.014897,0.000634,0.011103,0.212917,-0.125640,0.008662,0.029798,-0.020302,-0.163481,0.353566,0.219273,0.000000e+00,0.353566,-0.219273,0.000000,-0.353566,0.219273,0.000050,-0.353566,-0.219273,0.000000e+00,4.000850,-12.66800,36.21350,2.453700,-3.35677,36.0751,14.761500,-11.43940,82.5168,3.330220,-12.89850,30.6726,1,1,1,1
4,0,5,0.009452,-0.007070,0.504238,-0.015217,0.000045,0.010761,0.242628,-0.144926,0.010754,0.119397,-0.032874,-0.307524,0.353566,0.219273,0.000000e+00,0.353552,-0.219287,0.000296,-0.353572,0.219295,0.001007,-0.353566,-0.219273,9.969960e-05,0.677232,-12.62530,39.10510,2.200510,-3.08894,24.0039,16.722900,-12.89030,77.5639,0.799163,-12.18360,32.1531,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,196,1.734520,-1.197940,0.476707,-0.004933,-0.002012,-0.027549,0.037831,-0.026511,0.030403,0.139150,-0.034168,-0.047157,2.067480,-1.045250,3.311830e-02,2.104220,-1.483300,0.039428,1.447320,-1.035870,0.019043,1.355550,-1.376200,1.505400e-02,-15.507400,13.36760,56.88830,-0.485392,14.72290,63.3311,2.883620,8.17047,40.5672,-13.315800,8.10086,51.6205,1,1,1,1
196,0,197,1.734810,-1.198170,0.476830,-0.004988,-0.001642,-0.028191,0.023554,-0.011951,0.037063,0.091102,-0.042692,-0.030897,2.067480,-1.045250,3.311830e-02,2.104220,-1.483300,0.039428,1.447320,-1.035870,0.019043,1.355550,-1.376200,1.505400e-02,-14.271100,12.76920,56.77710,-1.898150,16.40520,60.2629,2.560820,8.99391,35.2156,-11.951000,8.11170,52.9538,1,1,1,1
197,0,198,1.735020,-1.198280,0.476961,-0.003156,-0.001210,-0.028648,0.012584,-0.003773,0.030435,0.050652,-0.034640,-0.016799,2.067480,-1.045250,3.311830e-02,2.104220,-1.483300,0.039428,1.447320,-1.035870,0.019043,1.355550,-1.376200,1.505400e-02,-15.140700,11.94100,52.59510,-4.697810,17.41460,61.5341,1.592200,9.59698,34.3275,-11.381800,8.09509,52.2396,1,1,1,1
198,0,199,1.735150,-1.198300,0.477102,0.000454,-0.000719,-0.028930,0.004728,-0.001604,0.011293,0.017356,-0.010979,-0.004737,2.067480,-1.045250,3.311830e-02,2.104220,-1.483300,0.039428,1.447320,-1.035870,0.019043,1.355550,-1.376200,1.505400e-02,-17.993500,10.89640,44.57940,-8.803580,17.79030,66.8919,0.015371,9.99253,37.6428,-11.561800,8.05261,49.5970,1,1,1,1


In [ ]:
'''
  Feet velocity
'''
# VLF
data=TerrainData.values
domain=14
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'V_LFx',footVel)

data=TerrainData.values
domain=15
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'V_LFy',footVel)

data=TerrainData.values
domain=16
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'V_LFz',footVel)

# VRF
data=TerrainData.values
domain=20
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'V_RFx',footVel)
data=TerrainData.values
domain=21
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'V_RFy',footVel)
data=TerrainData.values
domain=22
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'V_RFz',footVel)

# VLH
data=TerrainData.values
domain=26
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'V_LHx',footVel)
data=TerrainData.values
domain=27
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'V_LHy',footVel)
data=TerrainData.values
domain=28
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'V_LHz',footVel)

# VRH
data=TerrainData.values
domain=32
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'V_RHx',footVel)
data=TerrainData.values
domain=33
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'V_RHy',footVel)
data=TerrainData.values
domain=34
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'V_RHz',footVel)

'''
  Acceleration
'''

# AccLF
data=TerrainData.values
domain=17
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'a_LFx',footVel)
data=TerrainData.values
domain=18
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'a_LFy',footVel)
data=TerrainData.values
domain=19
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'a_LFz',footVel)

# AccRF
data=TerrainData.values
domain=26
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'a_RFx',footVel)
data=TerrainData.values
domain=27
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'a_RFy',footVel)
data=TerrainData.values
domain=28
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'a_RFz',footVel)

# AccLH
data=TerrainData.values
domain=35
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'a_LHx',footVel)
data=TerrainData.values
domain=36
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'a_LHy',footVel)
data=TerrainData.values
domain=37
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'a_LHz',footVel)

# AccRH
data=TerrainData.values
domain=44
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'a_RHx',footVel)
data=TerrainData.values
domain=45
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'a_RHy',footVel)
data=TerrainData.values
domain=46
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+3,'a_RHz',footVel)

# BaseA
data=TerrainData.values
domain=8
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+6,'aCOMx',footVel)
domain=9
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+6,'aCOMy',footVel)
domain=10
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+6,'aCOMz',footVel)
domain=11
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+6,'dWdx',footVel)
domain=12
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+6,'dWdy',footVel)
domain=13
footVel = (data[1:,domain]-data[:-1,domain])/0.01
footVel=np.insert(footVel, 0, 0., axis=0)
TerrainData.insert(domain+6,'dWdz',footVel)



TerrainData

,PathNum,NodeNum,baseX,baseY,baseZ,baseR,baseP,baseYaw,base_dX,base_dY,base_dZ,base_dR,base_dP,base_dYaw,aCOMx,aCOMy,aCOMz,dWdx,dWdy,dWdz,P_LFx,P_LFy,P_LFz,V_LFx,V_LFy,V_LFz,a_LFx,a_LFy,a_LFz,P_RFx,P_RFy,P_RFz,V_RFx,V_RFy,V_RFz,a_RFx,a_RFy,a_RFz,P_LHx,P_LHy,P_LHz,V_LHx,V_LHy,V_LHz,a_LHx,a_LHy,a_LHz,P_RHx,P_RHy,P_RHz,V_RHx,V_RHy,V_RHz,a_RHx,a_RHy,a_RHz,F_LFx,F_LFy,F_LFz,F_RFx,F_RFy,F_RFz,F_LHx,F_LHy,F_LHz,F_RHx,F_RHy,F_RHz,LF,RF,LH,RH
0,0,1,0.001688,-0.001070,0.123544,-0.003039,0.000163,0.002243,0.041294,-0.024394,0.003102,-0.022179,-0.001846,0.007847,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.087072,0.054000,3.751100e-39,0.00000,0.0000,0.000000e+00,0.000,0.00,0.000000e+00,0.087072,-0.054000,0.000000,0.00000,0.0000,0.000000,0.000,0.00,0.00000,-0.087072,0.054000,0.000047,0.00000,0.0000,0.000000,0.000,0.00,0.000000,-0.087072,-0.054000,3.159170e-40,0.00000,0.0000,0.000000e+00,0.000,0.00,0.000000e+00,2.581980,-2.76937,8.42542,0.783676,-0.92922,13.5573,2.622500,-1.83203,21.3714,2.212310,-3.35421,7.5617,0,0,0,0
1,0,2,0.005097,-0.003231,0.373127,-0.009178,0.000494,0.006774,0.124717,-0.073675,0.009370,-0.066987,-0.005575,0.023701,8.342290,-4.928110,0.626750,-4.480710,-0.372938,1.585351,0.262976,0.163091,1.275130e-39,17.59038,10.9091,-2.475970e-37,1759.038,1090.91,-2.475970e-35,0.262976,-0.163091,0.000000,17.59038,-10.9091,0.000000,1759.038,-1090.91,0.00000,-0.262976,0.163091,0.000142,-17.59038,10.9091,0.009487,-1759.038,1090.91,0.948655,-0.262976,-0.163091,1.073910e-40,-17.59038,-10.9091,-2.085260e-38,-1759.038,-1090.91,-2.085260e-36,7.798090,-8.36404,25.44650,2.366860,-2.80643,40.9458,7.920480,-5.53310,64.5460,6.681620,-10.13040,22.8379,0,0,0,0
2,0,3,0.006932,-0.004783,0.502204,-0.013175,0.000654,0.009759,0.182454,-0.107737,0.011312,-0.050916,-0.011678,-0.031935,5.773700,-3.406180,0.194228,1.607050,-0.610307,-5.563550,0.353566,0.219273,0.000000e+00,9.05900,5.6182,-1.275130e-37,-853.138,-529.09,1.200840e-35,0.353566,-0.219273,0.000000,9.05900,-5.6182,0.000000,-853.138,529.09,0.00000,-0.353566,0.219273,0.000145,-9.05900,5.6182,0.000285,853.138,-529.09,-0.920195,-0.353566,-0.219273,0.000000e+00,-9.05900,-5.6182,-1.073910e-38,853.138,529.09,1.011350e-36,8.366870,-11.71000,34.86590,2.944270,-3.63719,48.8534,11.992100,-8.74563,85.3883,7.137020,-13.38440,30.6945,1,1,1,1
3,0,4,0.007096,-0.005688,0.503320,-0.014897,0.000634,0.011103,0.212917,-0.125640,0.008662,0.029798,-0.020302,-0.163481,3.046300,-1.790300,-0.265048,8.071370,-0.862360,-13.154650,0.353566,0.219273,0.000000e+00,0.00000,0.0000,0.000000e+00,-905.900,-561.82,1.275130e-35,0.353566,-0.219273,0.000000,0.00000,0.0000,0.000000,-905.900,561.82,0.00000,-0.353566,0.219273,0.000050,0.00000,0.0000,-0.009487,905.900,-561.82,-0.977115,-0.353566,-0.219273,0.000000e+00,0.00000,0.0000,0.000000e+00,905.900,561.82,1.073910e-36,4.000850,-12.66800,36.21350,2.453700,-3.35677,36.0751,14.761500,-11.43940,82.5168,3.330220,-12.89850,30.6726,1,1,1,1
4,0,5,0.009452,-0.007070,0.504238,-0.015217,0.000045,0.010761,0.242628,-0.144926,0.010754,0.119397,-0.032874,-0.307524,2.971100,-1.928600,0.209258,8.959940,-1.257170,-14.404300,0.353566,0.219273,0.000000e+00,0.00000,0.0000,0.000000e+00,0.000,0.00,0.000000e+00,0.353552,-0.219287,0.000296,-0.00140,-0.0014,0.029603,-0.140,-0.14,2.96034,-0.353572,0.219295,0.001007,-0.00060,0.0022,0.095678,-0.060,0.22,10.516410,-0.353566,-0.219273,9.969960e-05,0.00000,0.0000,9.969960e-03,0.000,0.00,9.969960e-01,0.677232,-12.62530,39.10510,2.200510,-3.08894,24.0039,16.722900,-12.89030,77.5639,0.799163,-12.18360,32.1531,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,196,1.734520,-1.197940,0.476707,-0.004933,-0.002012,-0.027549,0.037831,-0.026511,0.030403,0.139150,-0.034168,-0.047157,-1.631690,2.032020,2.082056,-5.451

In [ ]:
TerrainData[' NodeNum']=(TerrainData[' NodeNum']-5)*10000000
TerrainData

,PathNum,NodeNum,baseX,baseY,baseZ,baseR,baseP,baseYaw,base_dX,base_dY,base_dZ,base_dR,base_dP,base_dYaw,aCOMx,aCOMy,aCOMz,dWdx,dWdy,dWdz,P_LFx,P_LFy,P_LFz,V_LFx,V_LFy,V_LFz,a_LFx,a_LFy,a_LFz,P_RFx,P_RFy,P_RFz,V_RFx,V_RFy,V_RFz,a_RFx,a_RFy,a_RFz,P_LHx,P_LHy,P_LHz,V_LHx,V_LHy,V_LHz,a_LHx,a_LHy,a_LHz,P_RHx,P_RHy,P_RHz,V_RHx,V_RHy,V_RHz,a_RHx,a_RHy,a_RHz,F_LFx,F_LFy,F_LFz,F_RFx,F_RFy,F_RFz,F_LHx,F_LHy,F_LHz,F_RHx,F_RHy,F_RHz,LF,RF,LH,RH
0,0,-40000000,0.001688,-0.001070,0.123544,-0.003039,0.000163,0.002243,0.041294,-0.024394,0.003102,-0.022179,-0.001846,0.007847,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.087072,0.054000,3.751100e-39,0.00000,0.0000,0.000000e+00,0.000,0.00,0.000000e+00,0.087072,-0.054000,0.000000,0.00000,0.0000,0.000000,0.000,0.00,0.00000,-0.087072,0.054000,0.000047,0.00000,0.0000,0.000000,0.000,0.00,0.000000,-0.087072,-0.054000,3.159170e-40,0.00000,0.0000,0.000000e+00,0.000,0.00,0.000000e+00,2.581980,-2.76937,8.42542,0.783676,-0.92922,13.5573,2.622500,-1.83203,21.3714,2.212310,-3.35421,7.5617,0,0,0,0
1,0,-30000000,0.005097,-0.003231,0.373127,-0.009178,0.000494,0.006774,0.124717,-0.073675,0.009370,-0.066987,-0.005575,0.023701,8.342290,-4.928110,0.626750,-4.480710,-0.372938,1.585351,0.262976,0.163091,1.275130e-39,17.59038,10.9091,-2.475970e-37,1759.038,1090.91,-2.475970e-35,0.262976,-0.163091,0.000000,17.59038,-10.9091,0.000000,1759.038,-1090.91,0.00000,-0.262976,0.163091,0.000142,-17.59038,10.9091,0.009487,-1759.038,1090.91,0.948655,-0.262976,-0.163091,1.073910e-40,-17.59038,-10.9091,-2.085260e-38,-1759.038,-1090.91,-2.085260e-36,7.798090,-8.36404,25.44650,2.366860,-2.80643,40.9458,7.920480,-5.53310,64.5460,6.681620,-10.13040,22.8379,0,0,0,0
2,0,-20000000,0.006932,-0.004783,0.502204,-0.013175,0.000654,0.009759,0.182454,-0.107737,0.011312,-0.050916,-0.011678,-0.031935,5.773700,-3.406180,0.194228,1.607050,-0.610307,-5.563550,0.353566,0.219273,0.000000e+00,9.05900,5.6182,-1.275130e-37,-853.138,-529.09,1.200840e-35,0.353566,-0.219273,0.000000,9.05900,-5.6182,0.000000,-853.138,529.09,0.00000,-0.353566,0.219273,0.000145,-9.05900,5.6182,0.000285,853.138,-529.09,-0.920195,-0.353566,-0.219273,0.000000e+00,-9.05900,-5.6182,-1.073910e-38,853.138,529.09,1.011350e-36,8.366870,-11.71000,34.86590,2.944270,-3.63719,48.8534,11.992100,-8.74563,85.3883,7.137020,-13.38440,30.6945,1,1,1,1
3,0,-10000000,0.007096,-0.005688,0.503320,-0.014897,0.000634,0.011103,0.212917,-0.125640,0.008662,0.029798,-0.020302,-0.163481,3.046300,-1.790300,-0.265048,8.071370,-0.862360,-13.154650,0.353566,0.219273,0.000000e+00,0.00000,0.0000,0.000000e+00,-905.900,-561.82,1.275130e-35,0.353566,-0.219273,0.000000,0.00000,0.0000,0.000000,-905.900,561.82,0.00000,-0.353566,0.219273,0.000050,0.00000,0.0000,-0.009487,905.900,-561.82,-0.977115,-0.353566,-0.219273,0.000000e+00,0.00000,0.0000,0.000000e+00,905.900,561.82,1.073910e-36,4.000850,-12.66800,36.21350,2.453700,-3.35677,36.0751,14.761500,-11.43940,82.5168,3.330220,-12.89850,30.6726,1,1,1,1
4,0,0,0.009452,-0.007070,0.504238,-0.015217,0.000045,0.010761,0.242628,-0.144926,0.010754,0.119397,-0.032874,-0.307524,2.971100,-1.928600,0.209258,8.959940,-1.257170,-14.404300,0.353566,0.219273,0.000000e+00,0.00000,0.0000,0.000000e+00,0.000,0.00,0.000000e+00,0.353552,-0.219287,0.000296,-0.00140,-0.0014,0.029603,-0.140,-0.14,2.96034,-0.353572,0.219295,0.001007,-0.00060,0.0022,0.095678,-0.060,0.22,10.516410,-0.353566,-0.219273,9.969960e-05,0.00000,0.0000,9.969960e-03,0.000,0.00,9.969960e-01,0.677232,-12.62530,39.10510,2.200510,-3.08894,24.0039,16.722900,-12.89030,77.5639,0.799163,-12.18360,32.1531,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,1910000000,1.734520,-1.197940,0.476707,-0.004933,-0.002012,-0.027549,0.037831,-0.026511,0.030403,0.139150,-0.034168,-0.04

In [ ]:
TerrainData[4:].to_csv('/content/drive/MyDrive/Colab Notebooks/Quadruped/myTest.csv',header=False)
TerrainData[4:]

,PathNum,NodeNum,baseX,baseY,baseZ,baseR,baseP,baseYaw,base_dX,base_dY,base_dZ,base_dR,base_dP,base_dYaw,aCOMx,aCOMy,aCOMz,dWdx,dWdy,dWdz,P_LFx,P_LFy,P_LFz,V_LFx,V_LFy,V_LFz,a_LFx,a_LFy,a_LFz,P_RFx,P_RFy,P_RFz,V_RFx,V_RFy,V_RFz,a_RFx,a_RFy,a_RFz,P_LHx,P_LHy,P_LHz,V_LHx,V_LHy,V_LHz,a_LHx,a_LHy,a_LHz,P_RHx,P_RHy,P_RHz,V_RHx,V_RHy,V_RHz,a_RHx,a_RHy,a_RHz,F_LFx,F_LFy,F_LFz,F_RFx,F_RFy,F_RFz,F_LHx,F_LHy,F_LHz,F_RHx,F_RHy,F_RHz,LF,RF,LH,RH
4,0,0,0.009452,-0.007070,0.504238,-0.015217,0.000045,0.010761,0.242628,-0.144926,0.010754,0.119397,-0.032874,-0.307524,2.971100,-1.928600,0.209258,8.959940,-1.257170,-14.404300,0.353566,0.219273,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00000,0.353552,-0.219287,0.000296,-0.0014,-0.0014,0.029603,-0.14,-0.14,2.96034,-0.353572,0.219295,0.001007,-0.0006,0.0022,0.095678,-0.06,0.22,10.516410,-0.353566,-0.219273,0.000100,0.0,0.0,0.009970,0.0,0.0,0.996996,0.677232,-12.62530,39.1051,2.200510,-3.088940,24.00390,16.722900,-12.89030,77.5639,0.799163,-12.18360,32.1531,1,1,1,1
5,0,10000000,0.014231,-0.008979,0.504937,-0.013987,-0.001173,0.008555,0.271506,-0.165743,0.018089,0.218817,-0.049809,-0.465380,2.887800,-2.081700,0.733500,9.942000,-1.693560,-15.785600,0.353566,0.219273,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00000,0.353522,-0.219317,0.000919,-0.0030,-0.0030,0.062323,-0.16,-0.16,3.27196,-0.353585,0.219340,0.003126,-0.0013,0.0045,0.211912,-0.07,0.23,11.623445,-0.353566,-0.219273,0.000310,0.0,0.0,0.020989,0.0,0.0,1.101938,-1.494260,-11.47650,43.7031,2.209690,-2.835050,12.71430,17.791400,-12.96760,70.3104,-0.321875,-11.21560,35.2941,1,1,1,1
6,0,20000000,0.017445,-0.011036,0.504784,-0.011663,-0.002670,0.002949,0.295276,-0.188694,0.031313,0.312819,-0.070205,-0.611283,2.377000,-2.295100,1.322360,9.400200,-2.039550,-14.590300,0.353566,0.219273,0.000111,0.0,0.0,0.011109,0.0,0.0,1.11093,0.353519,-0.219381,0.000963,-0.0003,-0.0064,0.004363,0.27,-0.34,-5.79604,-0.353657,0.219511,0.003601,-0.0072,0.0171,0.047536,-0.59,1.26,-16.437600,-0.353566,-0.219273,0.000324,0.0,0.0,0.001469,0.0,0.0,-1.952004,-2.476510,-10.07620,48.1853,2.335070,-2.168960,5.13033,17.336700,-11.94150,60.1424,-0.410741,-11.77310,39.1300,1,1,1,1
7,0,30000000,0.018860,-0.013261,0.503649,-0.008081,-0.004488,-0.006570,0.313171,-0.214100,0.051312,0.400586,-0.094581,-0.743433,1.789500,-2.540600,1.999860,8.776700,-2.437620,-13.215000,0.353566,0.219273,0.000350,0.0,0.0,0.023891,0.0,0.0,1.27818,0.353549,-0.219484,0.000340,0.0030,-0.0103,-0.062323,0.33,-0.39,-6.66856,-0.353796,0.219825,0.002185,-0.0139,0.0314,-0.141587,-0.67,1.43,-18.912300,-0.353566,-0.219273,0.000114,0.0,0.0,-0.020989,0.0,0.0,-2.245870,-2.090490,-8.38643,52.5345,2.594130,-1.028620,1.80973,15.129700,-9.64594,46.6213,0.687945,-14.08550,43.7654,1,1,1,1
8,0,40000000,0.022015,-0.015006,0.503376,-0.003957,-0.006145,-0.015385,0.327808,-0.239677,0.075977,0.464477,-0.122143,-0.844986,1.463700,-2.557700,2.466570,6.389100,-2.756190,-10.155300,0.353566,0.219273,0.000372,0.0,0.0,0.002150,0.0,0.0,-2.17409,0.354124,-0.219615,0.000000,0.0575,-0.0131,-0.033966,5.45,-0.28,2.83570,-0.354080,0.219274,0.001094,-0.0284,-0.0551,-0.109163,-1.45,-8.65,3.242400,-0.353566,-0.219273,0.000270,0.0,0.0,0.015515,0.0,0.0,3.650470,-1.155550,-7.95681,58.3654,2.498760,-0.013123,-0.00000,12.474900,-7.11298,34.1546,0.988579,-14.52980,50.2346,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,1910000000,1.734520,-1.197940,0.476707,-0.004933,-0.002012,-0.027549,0.037831,-0.026511,0.030403,0.139150,-0.034168,-0.047157,-1.631690,2.032020,2.082056,-5.451100,-1.241580,3.268960,2.067480,-1.045250,0.033118,0.0,0.0,0.000000,0.0,0.0,0.00000,2.104220,-1.483300,0.039428,0.0000,0.0000,0.000000,0.00,0.00,0.00000,1.447320,-1.035870,0.019043,0.0000,0.0000,0.000000,0.00,0.00,0.000000,1.355550,-1.376200,